#### __ResFinder__

Search carbapenemase gen occurrences using illumina reads directly

__Dependences__

- KMA (bioconda)
- BLAST (bioconda)
- resfinder (pip)

Databases

(in ~/group_storage/npilquinao/ISP/2carbapenemase/resfinder_databases/)

- git clone https://bitbucket.org/genomicepidemiology/resfinder_db/
- git clone https://bitbucket.org/genomicepidemiology/pointfinder_db/
- git clone https://bitbucket.org/genomicepidemiology/disinfinder_db/

Por alguna razon, pese a que resfinder es un programa en python, está construido solamente para ejecutarlo mediante CLI xD

In [17]:
!python -m resfinder --help

usage: __main__.py [-h] [-ifa INPUTFASTA] [-ifq INPUTFASTQ [INPUTFASTQ ...]]
                   [--nanopore] -o OUTPUTPATH [-j OUT_JSON] [-b BLASTPATH]
                   [-k KMAPATH] [-s SPECIES] [--ignore_missing_species]
                   [--output_aln] [-db_res DB_PATH_RES]
                   [-db_res_kma DB_PATH_RES_KMA] [-acq] [-ao ACQ_OVERLAP]
                   [-l MIN_COV] [-t THRESHOLD] [-d]
                   [-db_disinf DB_PATH_DISINF]
                   [-db_disinf_kma DB_PATH_DISINF_KMA] [-c]
                   [-db_point DB_PATH_POINT] [-db_point_kma DB_PATH_POINT_KMA]
                   [-g SPECIFIC_GENE [SPECIFIC_GENE ...]] [-u]
                   [-l_p MIN_COV_POINT] [-t_p THRESHOLD_POINT]
                   [--ignore_indels] [--ignore_stop_codons] [-v] [--pickle]

options:
  -h, --help            show this help message and exit
  -ifa INPUTFASTA, --inputfasta INPUTFASTA
                        Input fasta file.
  -ifq INPUTFASTQ [INPUTFASTQ ...], --inputfastq INPUTF

In [20]:
%%bash

cd /home/npilquinao/group_storage/npilquinao/ISP/2carbapenemase/samples/

python -m resfinder\
    --inputfastq\
        VA61_2022/61_22_S82_R1_001.fastq.gz\
        VA61_2022/61_22_S82_R2_001.fastq.gz\
    --outputPath ~/BEM-ISP/ResFinder/VA61_2022/\
    --acquired --disinfectant\
    --db_path_res ~/group_storage/npilquinao/ISP/2carbapenemase/resfinder_databases/resfinder_db/\
    --db_path_disinf ~/group_storage/npilquinao/ISP/2carbapenemase/resfinder_databases/disinfinder_db/\
    --db_path_point ~/group_storage/npilquinao/ISP/2carbapenemase/resfinder_databases/pointfinder_db/\

Error: KMA did not run as expected.
KMA finished with the following response:

Filename:	/home/npilquinao/group_storage/npilquinao/ISP/2carbapenemase/resfinder_databases/resfinder_db/aminoglycoside.length.b
Error: 2 (No such file or directory)



CalledProcessError: Command 'b'\ncd /home/npilquinao/group_storage/npilquinao/ISP/2carbapenemase/samples/\n\npython -m resfinder\\\n    --inputfastq\\\n        VA61_2022/61_22_S82_R1_001.fastq.gz\\\n        VA61_2022/61_22_S82_R2_001.fastq.gz\\\n    --outputPath ~/BEM-ISP/ResFinder/VA61_2022/\\\n    --acquired --disinfectant\\\n    --db_path_res ~/group_storage/npilquinao/ISP/2carbapenemase/resfinder_databases/resfinder_db/\\\n    --db_path_disinf ~/group_storage/npilquinao/ISP/2carbapenemase/resfinder_databases/disinfinder_db/\\\n    --db_path_point ~/group_storage/npilquinao/ISP/2carbapenemase/resfinder_databases/pointfinder_db/\\\n'' returned non-zero exit status 1.

__Desorden__

In [3]:
import resfinder

In [9]:
print(resfinder.__file__)

/home/npilquinao/miniconda3/envs/ISP/lib/python3.11/site-packages/resfinder/__init__.py


In [14]:
!ls /home/npilquinao/miniconda3/envs/ISP/lib/python3.11/site-packages/resfinder/
print("")
!cat /home/npilquinao/miniconda3/envs/ISP/lib/python3.11/site-packages/resfinder/run_resfinder.py

amr_abbreviations.md	  __init__.py  run_resfinder.py
cge			  __main__.py  species_abbreviations.md
environment_variables.md  __pycache__

#!/usr/bin/env python3
import io
import sys
import os
import subprocess
from argparse import ArgumentParser
import pickle
import json
import hashlib

from cgelib.output.result import Result
from cgelib.utils.loaders_mixin import LoadersMixin
from cgelib.utils.pliers_mixin import PliersMixin

from resfinder.cge.config import Config
from resfinder.cge.resfinder import ResFinder
from resfinder.cge.pointfinder import PointFinder
from resfinder.cge.output.std_results import ResFinderResultHandler
from resfinder.cge.output.std_results import PointFinderResultHandler

#  Modules used to create the extended ResFinder output (phenotype output)
from resfinder.cge.phenotype2genotype.isolate import Isolate
from resfinder.cge.phenotype2genotype.res_profile import PhenoDB
from resfinder.cge.phenotype2genotype.res_sumtable import ResSumTable
from resfinder.cge.phen